In [ ]:
!nvidia-smi

Mon Jul 26 10:22:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!wget -c "https://raw.githubusercontent.com/chef-transformer/chef-transformer/main/data/test-5000.csv" /content/

In [ ]:
%%capture
!pip install git+https://github.com/huggingface/datasets.git
!pip install git+https://github.com/huggingface/transformers.git

In [ ]:
%env LC_ALL=C.UTF-8
%env LANG=C.UTF-8
%env TRANSFORMERS_CACHE=/content/cache
%env HF_DATASETS_CACHE=/content/cache
%env CUDA_LAUNCH_BLOCKING=1

env: LC_ALL=C.UTF-8
env: LANG=C.UTF-8
env: TRANSFORMERS_CACHE=/content/cache
env: HF_DATASETS_CACHE=/content/cache
env: CUDA_LAUNCH_BLOCKING=1


In [ ]:
import numpy as np
import pandas as pd

from tqdm import tqdm

import os
import json

import torch
from datasets import load_dataset, load_metric

from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM

In [ ]:
model_name_or_path = "flax-community/t5-recipe-generation"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(model_name_or_path, device)

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path).to(device)

flax-community/t5-recipe-generation cuda


Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
prefix = "items: "
genkw = {
    "max_length": 512,
    "min_length": 64,
    "no_repeat_ngram_size": 3,
    "do_sample": True,
    "top_k": 60,
    "top_p": 0.95,
    "num_return_sequences": 5
}

In [ ]:
dataset = load_dataset("csv", data_files={"test": "/content/test-5000.csv"}, delimiter="\t")["test"]
dataset

Using custom data configuration default-96c47a923d37492d


0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /content/cache/csv/default-96c47a923d37492d/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


Dataset({
    features: ['inputs', 'targets'],
    num_rows: 5000
})

In [ ]:
def predict(batch):
    inputs = [prefix + inp for inp in batch["inputs"]]

    inputs = tokenizer(
        inputs, 
        max_length=256, 
        padding=True, 
        truncation=True, 
        return_tensors='pt'
    )
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)

    with torch.no_grad():
        output_ids = model.generate(
            input_ids=input_ids, 
            attention_mask=attention_mask,
            **genkw
        )

    outputs = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    batch["predicted"] = [outputs[i:i + genkw["num_return_sequences"]] for i in range(0, len(outputs), genkw["num_return_sequences"])]
    return batch

In [ ]:
# OR You can pass this step
# !wget -c "https://raw.githubusercontent.com/chef-transformer/chef-transformer/main/data/ChefTransformer_predicted.json" /content/predicted.json

In [ ]:
sample = dataset.select(range(5))
result = dataset.map(predict, batched=True, batch_size=8)
result

  0%|          | 0/625 [00:00<?, ?ba/s]

Dataset({
    features: ['inputs', 'predicted', 'targets'],
    num_rows: 5000
})

In [ ]:
df_list = []
for i in tqdm(range(len(result)), position=0):
    inputs, targets, predicted =  result["inputs"][i], result["targets"][i], result["predicted"][i]
    df_list.append({
        "input": inputs,
        "target": targets,
        "predicted": predicted,
    })

df = pd.DataFrame(df_list)
df.to_json("/content/predicted.json")
df.head()

100%|██████████| 5000/5000 [04:26<00:00, 18.76it/s]


,input,target,predicted
0,"margarine, sugar, eggs, sour cream, vanilla, c...",title: sour cream coffee cake ingredients: 1 c...,[title: sour cream pound cake ingredients: 1 c...
1,"peanut oil, brown onions, curry powder, cardam...",title: curried spinach onion rice ingredients:...,"[title: warm rice, spinach and almond curry in..."
2,"ground chuck, long shuttle noodles, spaghetti ...",title: cavatini casserole ingredients: 2 lb. g...,[title: goulash ingredients: 2 1/2 lb. ground ...
3,"dates, boiling water, baking mix, sugar, eggs,...",title: quick date nut bread ingredients: 1 1/2...,[title: date nut bread ingredients: 1 c. chopp...
4,"lemon juice, peaches, graham cracker pie crust",title: peach cream pie ingredients: 1 14 oz. c...,[title: peach pie ingredients: 1/3 c. lemon ju...


In [ ]:
df = pd.read_json("/content/t5_predicted.json")
df.head()

,input,target,predicted
0,"margarine, sugar, eggs, sour cream, vanilla, c...",title: sour cream coffee cake ingredients: 1 c...,[title: sour cream pound cake ingredients: 1 c...
1,"peanut oil, brown onions, curry powder, cardam...",title: curried spinach onion rice ingredients:...,"[title: warm rice, spinach and almond curry in..."
2,"ground chuck, long shuttle noodles, spaghetti ...",title: cavatini casserole ingredients: 2 lb. g...,[title: goulash ingredients: 2 1/2 lb. ground ...
3,"dates, boiling water, baking mix, sugar, eggs,...",title: quick date nut bread ingredients: 1 1/2...,[title: date nut bread ingredients: 1 c. chopp...
4,"lemon juice, peaches, graham cracker pie crust",title: peach cream pie ingredients: 1 14 oz. c...,[title: peach pie ingredients: 1/3 c. lemon ju...


In [ ]:
len(df.iloc[0]["predicted"])

5

In [ ]:
%%capture
!pip install -U nltk
!pip install jiwer
!pip install rouge_score

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords

import re
import math
from collections import Counter
import string
from tqdm import tqdm

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
stopwords = stopwords.words("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])
    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)
    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator
        
def text_to_vector(text):
    word = re.compile(r'\w+')
    words = word.findall(text)
    return Counter(words)

def get_result(content_a, content_b):
    text1 = content_a
    text2 = content_b
    vector1 = text_to_vector(text1)
    vector2 = text_to_vector(text2)
    cosine_result = get_cosine(vector1, vector2)
    return cosine_result

In [ ]:
# cosim_scores = []
# for index, row in tqdm(df.iterrows(), total=len(df), position=0):
#     target, predicted_list = row["target"], row["predicted"]

#     cosim_score = []
#     for j in range(len(predicted_list)):
#         cosim_score.append(get_result(target, predicted_list[j]))

#     cosim_scores.append(max(cosim_score))
#     # break


cosim_scores = []
for index, row in tqdm(df.iterrows(), total=len(df), position=0):
    target, predicted_list = row["target"], row["predicted"]

    target = " ".join([word.strip() for word in wordpunct_tokenize(target) if word not in string.punctuation and word not in stopwords and word])

    cosim_score = []
    for j in range(len(predicted_list)):
        predicted = " ".join([word.strip() for word in wordpunct_tokenize(predicted_list[j]) if word not in string.punctuation and word not in stopwords and word])
        cosim_score.append(get_result(target, predicted))

    cosim_scores.append(max(cosim_score))
    # break

100%|██████████| 5000/5000 [00:17<00:00, 279.11it/s]


In [ ]:
cosim_score = np.array(cosim_scores).mean()
print(f"Cosine similarity score: {cosim_score}")

Cosine similarity score: 0.7282591927547918


In [ ]:
wer = load_metric("wer")
wer_scores = []
for index, row in tqdm(df.iterrows(), total=len(df), position=0):
    target, predicted_list = row["target"], row["predicted"]

    wer_score = []
    for j in range(len(predicted_list)):
        score = wer.compute(references=[target], predictions=[predicted_list[j]])
        wer_score.append(score)

    wer_scores.append(min(wer_score))

    # break

100%|██████████| 5000/5000 [01:07<00:00, 74.54it/s]


In [ ]:
wer_score = np.array(wer_scores).mean()
print(f"WER score: {wer_score}")

WER score: 0.7613209678374205


In [ ]:
# rouge = load_metric("rouge")
# rouge_scores = []
# for index, row in tqdm(df.iterrows(), total=len(df), position=0):
#     target, predicted_list = row["target"], row["predicted"]

#     rouge_score = []
#     for j in range(len(predicted_list)):
#         score = rouge.compute(references=[target], predictions=[predicted_list[j]], use_stemmer=True)
#         rouge2 = score["rouge2"].mid.fmeasure
#         rouge_score.append(rouge2)

#     rouge_scores.append(max(rouge_score))

#     # break

 86%|████████▌ | 4304/5000 [1:05:14<10:39,  1.09it/s]

In [ ]:
rouge = load_metric("rouge")
rouge_scores = []
for index, row in tqdm(df.iterrows(), total=len(df), position=0):
    target, predictions = row["target"], row["predicted"]

    references = [target] * len(predictions)
    score = rouge.compute(references=references, predictions=predictions, use_stemmer=True)
    rouge2 = score["rouge2"].high.fmeasure
    rouge_scores.append(rouge2)

100%|██████████| 5000/5000 [30:37<00:00,  2.72it/s]


In [ ]:
rouge_score = np.array(rouge_scores).mean()
print(f"Rouge score: {rouge_score}") 

Rouge score: 0.24708236592873123


In [ ]:
import nltk
import nltk.translate.bleu_score as bleu
from nltk.translate.bleu_score import SmoothingFunction

import nltk.translate.gleu_score as gleu
import nltk.translate.meteor_score as meteor


def compute_bleu(recipe, refer):
    hyp = recipe
    refs = refer
    smoothie = SmoothingFunction().method4
    score_ref_a = bleu.sentence_bleu(refs, hyp, smoothing_function=smoothie)
    return score_ref_a


def compute_gleu(recipe, refer):
    hyp = recipe
    refs = refer
    score_ref_a = gleu.sentence_gleu(refs, hyp)
    return score_ref_a

def compute_meteor(recipe, refer):
    hyp = recipe
    refs = refer
    score_ref_a = meteor.meteor_score(refs, hyp)
    return score_ref_a

In [ ]:
bleu_scores = []
for index, row in tqdm(df.iterrows(), total=len(df), position=0):
    target, predicted = row["target"], row["predicted"]

    target = wordpunct_tokenize(target)
    predicted = [wordpunct_tokenize(p) for p in predicted]

    score = compute_bleu(target, predicted)
    bleu_scores.append(score)

    # break

100%|██████████| 5000/5000 [00:22<00:00, 220.66it/s]


In [ ]:
bleu_score = np.array(bleu_scores).mean()
print(f"BLEU score: {bleu_score}")

BLEU score: 0.324537926029933


In [ ]:
gleu_scores = []
for index, row in tqdm(df.iterrows(), total=len(df), position=0):
    target, predicted = row["target"], row["predicted"]

    target = wordpunct_tokenize(target)
    predicted = [wordpunct_tokenize(p) for p in predicted]

    score = compute_gleu(target, predicted)
    gleu_scores.append(score)

    # break

100%|██████████| 5000/5000 [00:17<00:00, 284.32it/s]


In [ ]:
gleu_score = np.array(gleu_scores).mean()
print(f"GLEU score: {gleu_score}")

GLEU score: 0.2624504703213403


In [ ]:
m_scores = []
for index, row in tqdm(df.iterrows(), total=len(df), position=0):
    target, predicted = row["target"], row["predicted"]

    score = compute_meteor(target, predicted)
    m_scores.append(score)

    # break

100%|██████████| 5000/5000 [04:21<00:00, 19.12it/s]


In [ ]:
m_score = np.array(m_scores).mean()
print(f"METEOR score: {m_score}")

METEOR score: 0.4150244019890512
